In [5]:
import paramiko

ip = "172.16.16.25"   # 원격 서버 IP
user = "root"
pw = "asd123!@"

SERVER_NFS_DIR = "/data/nfs_share"
CLIENT_NFS_DIR = "/mnt/nfs_share"
CLIENT_NFS_NI = "172.16.0.0" ## NFS를 허용해줄 CLIENT의 NFS IP입니다 
SERVER_NFS_IP = "172.16.16.100"

def cc(cmd):
    cli = paramiko.SSHClient()
    cli.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    cli.connect(ip, username=user, password=pw, port=22)
    stdin, stdout, stderr = cli.exec_command(cmd)
    out = stdout.read().decode()
    err = stderr.read().decode()
    cli.close()
    return out + err
##################################
#          SERVER_NFS            #
##################################
def nfs_install():
    print("=== NFS 설치 및 공유 디렉토리 생성 ===")
    cc("dnf -y install nfs-utils")
    cc("mkdir -p /mnt/nfs_share")

def server_nfs_mkdir():
    global SERVER_NFS_DIR
    SERVER_NFS_DIR = input("서버가 NFS로 공유해줄 디렉토리를 입력해주세요 ex. /data/nfs_share ")
    cmd =  f"""
mkdir -p {SERVER_NFS_DIR}
chown nobody:nogroup {SERVER_NFS_DIR}
chmod 777 {SERVER_NFS_DIR}
    """
    cc(cmd)
def server_nfs_set_exports_config():
  #  global CLIENT_NFS_NI
    CLIENT_NFS_NI = input("NFS 공유를 허용할 클라이언트의 NI를 입력해주세요 (예: 172.16.0.0/16): ")
    print(f"클라이언트 {CLIENT_NFS_NI} NI 대역에게 읽기/쓰기 권한을 부여합니다.")
    
    cc(f"echo '{SERVER_NFS_DIR} {CLIENT_NFS_NI}(rw,sync,no_subtree_check)' >> /etc/exports")
    cc("exportfs -ra")


def nfs_service():
    print("=== NFS 서비스 시작 ===")
    cc("systemctl enable nfs-server")
    cc("systemctl start nfs-server")
    cc("exportfs -v")  # 공유 확인
    print("NFS 서비스 활성화 완료")

##################################
#          CLIENT_NFS            #
##################################

def client_install():
    print("=== NFS 클라이언트 마운트 ===")
    cc("dnf -y install nfs-utils")   # 클라이언트에도 설치 필요

def client_nfs_mount():
    global SERVER_NFS_IP
    SERVER_NFS_IP = input("nfs server ip 입력")
    global CLIENT_NFS_DIR
    global SERVER_NFS_DIR
    SERVER_NFS_DIR = input("server의 nfs 디렉토리 입력 ex. /data/nfs_share ")
    CLIENT_NFS_DIR = input("client mount 할 디렉토리 입력 ex. /mnt/nfs_share ")
    cc(f"mkdir -p {CLIENT_NFS_DIR}")
    cc(f"mount -t nfs {SERVER_NFS_IP}:{SERVER_NFS_DIR} {CLIENT_NFS_DIR}")
    print("NFS 마운트 완료")

def server_nfs():
    nfs_install()
    server_nfs_mkdir()
    server_nfs_set_exports_config()
    nfs_service()
def client_nfs():
    client_install()
    client_nfs_mount()
#server_nfs()
client_nfs()


=== NFS 클라이언트 마운트 ===


nfs server ip 입력 172.16.16.100
server의 nfs 디렉토리 입력 ex. /data/nfs_share  /data/z
client mount 할 디렉토리 입력 ex. /mnt/nfs_share  /mnt/z


NFS 마운트 완료
